# Behavioural Simulation of GA-inspired Quantum Search

GA-inspired Quantum Search can be simulated easily via classical computation because of the nature of doing successive of Grover's algorithm without the need to do complex multiple qubit gate. 

Since we want to see the our quantum states easily we will need to visualize the states. So we import the visualization stuff and other required packages.

In [ ]:
import matplotlib
import random
import math

import itertools
import bisect

%matplotlib inline

Before we begin, we need to implement the state class to represents our quantum states for easy checking and easy to use.

We will split states into different parts and for each part the lower the index the more significant the bit is.

In [ ]:
class State:
    
    def __init__(self, n):
        self.len = n
        self.state = [([0] * n, 1)]
        
    def get_mean(self, n=None):
        if n is None:
            n = self.len
        mean = sum([amp for st, amp in self.state])/(2**n)
        return mean
        
    def measure(self, a, b):
        n = self.len
        if a > b or a < 0 or b > n:
            return False
        cum_prob = [amp * amp for st, amp in self.state]

        for i in range(1, len(self.state)):
            cum_prob[i] += cum_prob[i - 1]
        r = random.uniform(0, 1)
        print('random = {}'.format(r))
        idx = bisect.bisect_left(cum_prob, r)
        print('id = {}'.format(idx))
        result = self.state[idx][0][a:b]
        self.state = list(filter(lambda st: st[0][a:b] == result, self.state))
        self.__normalize__()
        return result
    
    def set_value(self, a, b, val):
        # check whether val can fit into (a,b) range
        bstr = '{0:b}'.format(val).zfill(b - a)
        if len(bstr) > b - a:
            raise valueError('value is too large')
        bstr_l = [int(x) for x in bstr]
        self.measure(a, b)
        for i in range(len(self.state)):
            self.state[i][0][a:b] = bstr_l
            
        self.__cleanup__()
            
    
    def hadamard(self, a, b):
        n = self.len
        if a > b or a < 0 or b > n:
            raise valueError("can't hadamard this right now: reason 1")
        if list(filter(lambda st: st[0][a:b] != ([0]*(b-a)), self.state)):
            raise valueError("can't hadamard this right now: reason 2")
        temp = []
        for st, amp in self.state:
            for i in range(2**(b-a)):
                nst = st[:]
                nst[a:b] = [int(x) for x in '{0:b}'.format(i).zfill(b-a)]
                temp.append((nst, amp * 1 / math.sqrt(2**(b-a))))
        self.state = temp
        
    def __normalize__(self):
        renorm_factor = math.sqrt(sum([amp*amp for st, amp in self.state]))
        self.state = [(st, amp/renorm_factor) for st, amp in self.state]

    def __cleanup__(self):
        self.state = sorted([(st, sum(j for i, j in amp_group)) for st, amp_group in itertools.groupby(sorted(self.state), key=lambda x: x[0])])
        

In [ ]:
st = State(4)
st.hadamard(0, 4)
st.set_value(2, 4, 3)

print(a)
print(st.state)
print(st.get_mean(4))

Structure of the ... (i.e. outline of the algorithm and how it will be used)

In [ ]:
def comparator(state, bound_a, bound_b):
    """
    bound is [start, end)
    """
    n = len(state)
    
    if type(bound_a) is not tuple or type(bound_b) is not tuple:
        raise ValueError('invalid input: not tuples')

#     if type(state) is not State:
#         raise ValueError('input is not a valid state')
        
    a, b = bound_a
    c, d = bound_b
    
    if b - a != d - c:
        raise ValueError("input qreg doesn't have the same size = {}, {}".format(b-a, d-c))
        
    if a < 0 or b > n or c < 0 or d > n:
        raise ValueError('index out of bound')

    return state[a:b] > state[c:d]
    
    
    
def grover(states, Uf, input_bound, bounds):
    """
    n: the number of qubit
    states: [(state, amplitude)]
    input_bound: (a, b) 
    bounds: [(start, end)] 
    Uf: function that output 0, 1
    """
    n = states.len
    bds = [input_bound] + bounds
    # flip state that has Uf(x) = 1
    # Todo: make it more general
    states.state = [(st, -amp) if Uf(st, bds[0], bds[1]) else (st, amp) for (st, amp) in states.state]
    print(states.state)
    # inverse around the mean
    mean = states.get_mean(input_bound[1] - input_bound[0])
    print('mean = {}'.format(mean))
    states.state = [(st, 2*mean-amp) for (st, amp) in states.state]

In [ ]:
st = State(4)
st.hadamard(0, 4)
print(st.state)
st.set_value(2, 4, 2)
grover(st, comparator, (0,2), [(2, 4)])

print(st.state)